In [29]:
import pywt
from matplotlib import pyplot
%matplotlib inline
import numpy
from PIL import Image
import urllib.request
import io

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.set_printoptions(precision=6, sci_mode=False)

In [29]:
# pyright: reportMissingImports=false
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

from src.shared.gaussian import Gaussian
from src.shared.faces import make_cube_faces
from src.shared.sides import (    
    to_vertices,
    make_phi_theta,
    sphered_vertices,
    to_spherical,
)

class Coarse(nn.Module):
    def __init__(self, ns, kernel=3, sigma=1, r=0.5, ch=32, bias=True):
        super(Coarse, self).__init__()
        n = ns[-1]
        self.n = n
        phi, theta = make_phi_theta(n)
        self.register_buffer('phi', phi)
        self.register_buffer('theta', theta)
        self.register_buffer('faces', make_cube_faces(n))
        
        self.abc = nn.Parameter(torch.zeros(1, 3, 1, 1))
        self.radii = nn.ParameterList([
            nn.Parameter(torch.zeros(6, 3, l, l)) for l in ns])        
        

        self.gaussian =  Gaussian(kernel, sigma=sigma)
   
    # def get_ellipsoidal(self, radii, abc):
    #     x = (abc[0] + radii[:, 0]) * torch.sin(self.theta) * torch.cos(self.phi)
    #     y = (abc[1] + radii[:, 1]) * torch.sin(self.theta) * torch.sin(self.phi)
    #     z = (abc[2] + radii[:, 2]) * torch.cos(self.theta)
    #     return torch.stack((x, y, z), dim=1)   
    
    def get_ellipsoidal(self, radii):
        x = radii[:, 0] * torch.sin(self.theta) * torch.cos(self.phi)
        y = radii[:, 1] * torch.sin(self.theta) * torch.sin(self.phi)
        z = radii[:, 2] * torch.cos(self.theta)
        return torch.stack((x, y, z), dim=1) 
    
    def scale(_, t, sz):
        return F.interpolate(t, sz, mode='bilinear', align_corners=True)
    
    def forward(self): 
        radii = self.gaussian(self.radii[0])
        for high in self.radii[1:]:            
            radii = self.scale(radii, high.size(-1)) + self.gaussian(high)
            
        radii = torch.sigmoid(radii +self.abc)
        #abc = torch.sigmoid(radii + self.abc) 
        stacked = self.get_ellipsoidal(radii)
        vert = to_vertices(stacked)
        return vert, self.faces, radii, stacked

    def save(self, filename):
        torch.save(self.radii.data, filename)

    def load(self, filename):
        self.radii = nn.Parameter(torch.load(filename))

coarse = Coarse([8, 12, 16, 24, 32])        
vert , _, _, stacked = coarse()
vert.shape,stacked.shape

(torch.Size([6144, 3]), torch.Size([6, 3, 32, 32]))

In [4]:
ns = [4, 6, 8]
OrderedDict({
    str(n): nn.Parameter(torch.zeros(6, 3, n, n)) for n in ns
})

OrderedDict([('4', Parameter containing:
              tensor([[[[0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.]],
              
                       [[0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.]],
              
                       [[0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.]]],
              
              
                      [[[0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.]],
              
                       [[0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.],
                        [0., 0., 0., 0.]],
              
                       

In [6]:
radii = nn.ParameterList([nn.Parameter(torch.zeros(6, 3, n, n)) for n in ns])
radii

ParameterList(
    (0): Parameter containing: [torch.FloatTensor of size 6x3x4x4]
    (1): Parameter containing: [torch.FloatTensor of size 6x3x6x6]
    (2): Parameter containing: [torch.FloatTensor of size 6x3x8x8]
)

In [25]:
self = coarse
radii = self.gaussian(self.radii[0])
for high in self.radii[1:]:   
    #print( self.scale(radii, high.size(-1)).shape,  self.gaussian(high).shape)
    radii = self.scale(radii, high.size(-1)) + self.gaussian(high)

torch.Size([6, 3, 12, 12]) torch.Size([6, 3, 12, 12])
torch.Size([6, 3, 16, 16]) torch.Size([6, 3, 16, 16])
torch.Size([6, 3, 24, 24]) torch.Size([6, 3, 24, 24])
torch.Size([6, 3, 32, 32]) torch.Size([6, 3, 32, 32])


In [17]:
 self.scale(radii, high.size(-1)).shape

torch.Size([6, 3, 8, 8])

In [24]:
radii[1:].shape

torch.Size([5, 3, 8, 8])

In [21]:
self.gaussian(high)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [3, 1, 3, 3], but got 3-dimensional input of size [3, 10, 10] instead